In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
import pandas as pd
import seaborn as sns
import numpy as np
import squarify
import pandas as pd
import rpy2.robjects as ro
import matplotlib

from collections import defaultdict
from copy import deepcopy
from scipy import stats
from colour import Color
from matplotlib import pyplot as plt
from pathlib import Path
from upsetplot import UpSet, from_contents
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from matplotlib_venn import venn3, venn3_circles

In [ ]:
src_path: str = "/home/uziel/Development/biopipes/src"
sys.path.insert(0, src_path)

In [ ]:
from r_wrappers.utils import rpy2_df_to_pd_df, pd_df_to_rpy2_df
from r_wrappers.deseq2 import vst_transform
from r_wrappers.limma import *
from components.functional_analysis.orgdb import OrgDB

org_db = OrgDB("Homo sapiens")
root = Path("/home/uziel/storage")
pcta_wcdt_root = root.joinpath("PCTA_WCDT_FOLH1")
tcga_root = root.joinpath("TCGA_PRAD_MethArray_FOLH1")
wcdt_root = root.joinpath("WCDT_MCRPC_FOLH1")
palette_str = "flare"

sns.set_theme(style="whitegrid", palette=palette_str)
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Ubuntu Mono"
plt.rcParams["font.monospace"] = "Ubuntu Mono"

In [ ]:
pcta_wcdt_figs_path = pcta_wcdt_root.joinpath("manuscript_figures")
pcta_wcdt_figs_path.mkdir(parents=True, exist_ok=True)
pcta_wcdt_add_files_path = pcta_wcdt_root.joinpath("additional_files")
pcta_wcdt_add_files_path.mkdir(parents=True, exist_ok=True)

In [ ]:
pcta_wcdt_annot_df = pd.read_csv(
    pcta_wcdt_root.joinpath("data").joinpath("samples_annotation_FOLH1.csv"),
    index_col=0,
)
tcga_annot_df = pd.read_csv(
    tcga_root.joinpath("data").joinpath("samples_annotation_FOLH1.csv"), index_col=0
)
wcdt_annot_df = pd.read_csv(
    wcdt_root.joinpath("data").joinpath("samples_annotation_FOLH1.csv"), index_col=0
)

## Figure 1: Workflow

A workflow schematic showing our method from start to finish. Kind of a graphical abstract, it must be visually attractive and contain few technical details. **This should give the reader an overview of the whole methodology.**


![Methodology Workflow](../../../docs/1_workflow.svg)


## Figure 2: Data

Focus on PCTA+WCDT (PCTA+), but mention that a subset of samples has DNA methylation or WGBS data available. **This should show a good summary of the data used in the study.**

1. (**A**) PCA of all samples (before batch correction)
2. (**B**) Treemap with dataset composition of PCTA+
3. (**C**) Treemap of tissue origin in metastatic samples
4. (**D**) PCA of all samples (after batch correction)
5. (**E**) PSMA levels of each group in a raincloud plot (prim)
6. (**F**) PSMA levels of each group in a raincloud plot (met)

In [ ]:
with pd.ExcelWriter(pcta_wcdt_add_files_path / "additional_file_1.xlsx") as writer:
    pcta_wcdt_annot_df.to_excel(writer, sheet_name="1A. PCTA+ Annotations")
    tcga_annot_df.to_excel(writer, sheet_name="1B. TCGA-PRAD Annotations")
    wcdt_annot_df.to_excel(writer, sheet_name="1C. WCDT-MCRPC Annotations")

### 2A: PCA of all samples (before batch correction)

In [ ]:
counts_df = pd.read_csv(
    root.joinpath("PCTA_WCDT").joinpath("data").joinpath("raw_counts.csv"), index_col=0
)
print(counts_df.shape)
counts_df_wo_batch_effects = pd.read_csv(
    root.joinpath("PCTA_WCDT")
    .joinpath("data")
    .joinpath("raw_counts_wo_batch_effects.csv"),
    index_col=0,
)
print(counts_df.shape)

In [ ]:
common_samples = pcta_wcdt_annot_df.index.intersection(counts_df.columns)
annot_df_contrasts = pcta_wcdt_annot_df.loc[common_samples, :]
counts_df = counts_df.loc[:, common_samples]
counts_df_wo_batch_effects = counts_df_wo_batch_effects.loc[:, common_samples]

In [ ]:
vst_df = rpy2_df_to_pd_df(
    vst_transform(
        ro.r("as.matrix")(pd_df_to_rpy2_df(counts_df.loc[counts_df.mean(axis=1) > 1]))
    )
)
vst_wo_batch_effects_df = rpy2_df_to_pd_df(
    vst_transform(
        ro.r("as.matrix")(
            pd_df_to_rpy2_df(
                counts_df_wo_batch_effects.loc[
                    counts_df_wo_batch_effects.mean(axis=1) > 10
                ]
            )
        )
    )
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)

pca = PCA(n_components=2)
pca_comps = pca.fit_transform(vst_df.transpose().values)
palette = {
    "norm": "#9ACD32",
    "prim": "#4A708B",
    "met": "#8B3A3A",
}

annot_df_contrasts["Sample Type"] = annot_df_contrasts["sample_type"]
annot_df_contrasts["Library Type"] = annot_df_contrasts["library_type"].map(
    {"total_rna": "Total RNA", "hybrid_capture": "Hybrid Capture", "poly_a": "Poly(A)"}
)
sns.scatterplot(
    x=pca_comps.T[0],
    y=pca_comps.T[1],
    hue=annot_df_contrasts["Sample Type"],
    palette=palette,
    style=annot_df_contrasts["Library Type"],
    ax=ax,
)

plt.legend(loc="upper right")
# plt.suptitle("PCA of PCTA+WCDT samples (before batch correction)", fontsize=18)
pca_variances = pca.explained_variance_ratio_
ax.set_xlabel(f"PC1 ({pca_variances[0] * 100:.2f}%)", fontsize=14)
ax.set_ylabel(f"PC2 ({pca_variances[1] * 100:.2f}%)", fontsize=14)
plt.tight_layout()
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2A_pca_before_combat.pdf"), bbox_inches="tight"
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2A_pca_before_combat.svg"), bbox_inches="tight"
)

### 2B: Treemap with dataset composition of PCTA+

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)

data = pcta_wcdt_annot_df
sizes = data["dataset"].value_counts()

labels = [f"{label}\n({value}/{len(data)})" for label, value in sizes.items()]

norm = matplotlib.colors.Normalize(vmin=sizes.values.min(), vmax=sizes.values.max())
cmap = sns.color_palette("flare_r", as_cmap=True)
colors = [cmap(norm(value)) for value in sizes.values]

squarify.plot(
    sizes=sizes.values,
    label=labels,
    alpha=0.8,
    color=colors,
    text_kwargs={"fontsize": 10},
)
# plt.title("PCTA+ dataset composition")
plt.axis("off")
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("2B_dataset_origin.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("2B_dataset_origin.svg"), bbox_inches="tight")

### 2C: Treemap of tissue origin in metastatic samples

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)

data = pcta_wcdt_annot_df[pcta_wcdt_annot_df["sample_type"] == "met"]
data.loc[:, "tumor_tissue_site"] = data["tumor_tissue_site"].apply(
    lambda x: x.replace("_", " ").title() if isinstance(x, str) else x
)
sizes = data["tumor_tissue_site"].value_counts()
sizes_clean = sizes.drop(sizes[sizes < 7].index)
sizes_clean["Other"] = sum(sizes[sizes < 7])

labels = [f"{label}\n({value}/{len(data)})" for label, value in sizes_clean.items()]

norm = matplotlib.colors.Normalize(
    vmin=sizes_clean.values.min(), vmax=sizes_clean.values.max()
)
cmap = sns.color_palette("flare_r", as_cmap=True)
colors = [cmap(norm(value)) for value in sizes_clean.values]

squarify.plot(
    sizes=sizes_clean.values,
    label=labels,
    alpha=0.8,
    color=colors,
    text_kwargs={"fontsize": 10},
)
# plt.title("Origin tissue site of metastatic samples in PCTA+")
plt.axis("off")
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("2C_tissue_origin.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("2C_tissue_origin.svg"), bbox_inches="tight")

### 2D: PCA of all samples (after batch correction)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=100)

pca = PCA(n_components=2)
pca_comps = pca.fit_transform(vst_wo_batch_effects_df.transpose().values)
palette = {
    "norm": "#9ACD32",
    "prim": "#4A708B",
    "met": "#8B3A3A",
}
annot_df_contrasts["Sample Type"] = annot_df_contrasts["sample_type"]
annot_df_contrasts["Library Type"] = annot_df_contrasts["library_type"].map(
    {"total_rna": "Total RNA", "hybrid_capture": "Hybrid Capture", "poly_a": "Poly(A)"}
)
sns.scatterplot(
    x=pca_comps.T[0],
    y=pca_comps.T[1],
    hue=annot_df_contrasts["Sample Type"],
    palette=palette,
    style=annot_df_contrasts["Library Type"],
    ax=ax,
)

plt.legend(loc="lower right")
# plt.suptitle("PCA of PCTA+WCDT samples (after batch correction)", fontsize=18)
pca_variances = pca.explained_variance_ratio_
ax.set_xlabel(f"PC1 ({pca_variances[0] * 100:.2f}%)", fontsize=14)
ax.set_ylabel(f"PC2 ({pca_variances[1] * 100:.2f}%)", fontsize=14)
plt.tight_layout()
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2D_pca_after_combat.pdf"), bbox_inches="tight"
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2D_pca_after_combat.svg"), bbox_inches="tight"
)

### 2E: PSMA levels of each group in a raincloud plot (prim)

In [ ]:
data_df = pcta_wcdt_annot_df[pcta_wcdt_annot_df["sample_type"] == "prim"]

data_x = []
folh1_levels = []
for folh1_level in ("high", "mid", "low"):
    x = data_df[data_df["FOLH1_level"] == folh1_level]["FOLH1_VST"]
    data_x.append(x)
    folh1_levels.append(folh1_level + f"\n({len(x)})")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=100)

# Boxplot data
bp = ax.boxplot(data_x, patch_artist=True, vert=False, widths=0.1)

# Change to the desired color and add transparency
for patch in bp["boxes"]:
    # patch.set_facecolor(color)
    patch.set_alpha(0.4)

# Violinplot data
vp = ax.violinplot(
    data_x,
    # points=500,
    showmeans=False,
    showextrema=False,
    showmedians=False,
    vert=False,
)

for idx, b in enumerate(vp["bodies"]):
    # Get the center of the plot
    m = np.mean(b.get_paths()[0].vertices[:, 0])

    # Modify it so we only see the upper half of the violin plot
    b.get_paths()[0].vertices[:, 1] = np.clip(
        b.get_paths()[0].vertices[:, 1], idx + 1, idx + 2
    )

# Scatterplot data
for idx, features in enumerate(data_x):
    # Add jitter effect so the features do not overlap on the y-axis
    y = np.full(len(features), idx + 0.8)
    idxs = np.arange(len(y))
    out = y.astype(float)
    out.flat[idxs] += np.random.uniform(low=-0.05, high=0.05, size=len(idxs))
    y = out
    plt.scatter(features, y, s=1)

plt.yticks(np.arange(1, len(folh1_levels) + 1, 1), folh1_levels)
plt.ylabel("FOLH1 group", fontsize=14)
plt.xlabel("FOLH1 RNA Expression (VST-transformed)", fontsize=14)
# plt.suptitle("Optimal stratification of primary samples in PCTA+", fontsize=18)
plt.tight_layout()
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2E_prim_psma_rainclouds.pdf"), bbox_inches="tight"
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2E_prim_psma_rainclouds.svg"), bbox_inches="tight"
)

### 2F: PSMA levels of each group in a raincloud plot (met)

In [ ]:
data_df = pcta_wcdt_annot_df[pcta_wcdt_annot_df["sample_type"] == "met"]

data_x = []
folh1_levels = []
for folh1_level in ("high", "mid", "low"):
    x = data_df[data_df["FOLH1_level"] == folh1_level]["FOLH1_VST"]
    data_x.append(x)
    folh1_levels.append(folh1_level + f"\n({len(x)})")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4), dpi=100)

# Boxplot data
bp = ax.boxplot(data_x, patch_artist=True, vert=False, widths=0.1)

# Change to the desired color and add transparency
for patch in bp["boxes"]:
    # patch.set_facecolor(color)
    patch.set_alpha(0.4)

# Violinplot data
vp = ax.violinplot(
    data_x,
    # points=500,
    showmeans=False,
    showextrema=False,
    showmedians=False,
    vert=False,
)

for idx, b in enumerate(vp["bodies"]):
    # Get the center of the plot
    m = np.mean(b.get_paths()[0].vertices[:, 0])

    # Modify it so we only see the upper half of the violin plot
    b.get_paths()[0].vertices[:, 1] = np.clip(
        b.get_paths()[0].vertices[:, 1], idx + 1, idx + 2
    )

# Scatterplot data
for idx, features in enumerate(data_x):
    # Add jitter effect so the features do not overlap on the y-axis
    y = np.full(len(features), idx + 0.8)
    idxs = np.arange(len(y))
    out = y.astype(float)
    out.flat[idxs] += np.random.uniform(low=-0.05, high=0.05, size=len(idxs))
    y = out
    plt.scatter(features, y, s=1)

plt.yticks(np.arange(1, len(folh1_levels) + 1, 1), folh1_levels)
plt.ylabel("FOLH1 group", fontsize=14)
plt.xlabel("FOLH1 RNA Expression (VST-transformed)", fontsize=14)
# plt.suptitle("Optimal stratification of metastatic samples in PCTA+", fontsize=18)
plt.tight_layout()

plt.savefig(
    pcta_wcdt_figs_path.joinpath("2F_met_psma_rainclouds.pdf"), bbox_inches="tight"
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("2F_met_psma_rainclouds.svg"), bbox_inches="tight"
)

## Figure 3: Differential analyses

This figure should summarize the gene sets obtained from D-GSVA and GSEA (DE & DM).

1. (**A**) Barplot of D-GSVA gene sets for category H (prim over/under)
2. (**B**) Barplot of D-GSVA gene sets for category H (met over/under)
3. (**C**) Starburst plot (prim)
4. (**D**) Barplot of GSEA (DE) gene sets for category H (prim over/under)
5. (**E**) Barplot of GSEA (DM) gene sets for category H (prim over/under)
6. (**F**) Starburst plot (met)
7. (**G**) Barplot of GSEA (DE) gene sets for category H (met over/under)
8. (**H**) Barplot of GSEA (DM) gene sets for category H (met over/under)

In [ ]:
writer_2 = pd.ExcelWriter(pcta_wcdt_add_files_path / "additional_file_2.xlsx")

### 3A: Barplot of D-GSVA gene sets for category H (prim over/under)

In [ ]:
dgsva_h_df = pd.read_csv(
    pcta_wcdt_root.joinpath("diff_gsva")
    .joinpath("H")
    .joinpath(
        "sample_type_met+prim_FOLH1_level_prim_high+prim_low_"
        "_prim_high_vs_prim_low_top_table_padj_0_05_all_0_0.csv"
    ),
    index_col=0,
)
dgsva_h_df.index = [
    idx.replace("HALLMARK_", "").replace("_", " ") for idx in dgsva_h_df.index
]

In [ ]:
dgsva_h_df.to_excel(writer_2, sheet_name="2A. D-GSVA H (PRIM)")

In [ ]:
data = dgsva_h_df["log2FoldChange"].sort_values(ascending=False).round(2).to_frame()

fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.barplot(data=data, x="log2FoldChange", y=data.index, palette=palette_str)
for container in ax.containers:
    ax.bar_label(container, padding=5)

ax.set_ylabel("Gene Sets", fontsize=20)
plt.yticks(fontsize=16)
ax.set_xlabel("$Log_2$ Fold Change", fontsize=20)
ax.set_xlim([ax.get_xlim()[0] * 1.2, ax.get_xlim()[1] * 1.2])
# plt.title(
#     "D-GSVA Analysis on MSigDB H Collection (Prim. High vs Prim. low)",
#     fontsize=20,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3A_dgsva_h_prim.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3A_dgsva_h_prim.svg"), bbox_inches="tight")

### 3B: Barplot of D-GSVA gene sets for category H (met over/under)

In [ ]:
dgsva_h_df = pd.read_csv(
    pcta_wcdt_root.joinpath("diff_gsva")
    .joinpath("H")
    .joinpath(
        "sample_type_met+prim_FOLH1_level_met_high+met_low_"
        "_met_high_vs_met_low_top_table_padj_0_05_all_0_0.csv"
    ),
    index_col=0,
)
dgsva_h_df.index = [
    idx.replace("HALLMARK_", "").replace("_", " ") for idx in dgsva_h_df.index
]

In [ ]:
dgsva_h_df.to_excel(writer_2, sheet_name="2B. D-GSVA H (MET)")

In [ ]:
data = dgsva_h_df["log2FoldChange"].sort_values(ascending=False).round(2).to_frame()

fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.barplot(data=data, x="log2FoldChange", y=data.index, palette=palette_str)
for container in ax.containers:
    ax.bar_label(container, padding=5)

ax.set_ylabel("Gene Sets", fontsize=20)
plt.yticks(fontsize=16)
ax.set_xlabel("$Log_2$ Fold Change", fontsize=20)
ax.set_xlim([ax.get_xlim()[0] * 1.2, ax.get_xlim()[1] * 1.2])
# plt.title(
#     "D-GSVA Analysis on MSigDB H Collection (Met. High vs Met. low)",
#     fontsize=20,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3B_dgsva_h_met.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3B_dgsva_h_met.svg"), bbox_inches="tight")

### 3C: Starburst plot (prim)

In [ ]:
label_colors = {
    "Not relevant": "gray",
    "Up regulated": "#94090D",
    "Up regulated & Hyper methylated": "#450003",
    "Up regulated & Hypo methylated": "#FF1D23",
    "Down regulated": "#A7E7F6",
    "Down regulated & Hyper methylated": "#318EAD",
    "Down regulated & Hypo methylated": "#225A76",
    "Hyper methylated": "#F06E19",
    "Hypo methylated": "#3FCEF6",
}

In [ ]:
df = pd.read_csv(
    (
        pcta_wcdt_root.joinpath("deseq2").joinpath(
            "sample_type_met+prim_FOLH1_level_prim_high+prim_low__"
            "prim_high_vs_prim_low_deseq_results_unique.csv"
        )
    ),
    index_col=0,
).set_index("SYMBOL")

In [ ]:
diff_expr_df = pd.read_csv(
    (
        pcta_wcdt_root.joinpath("deseq2").joinpath(
            "sample_type_met+prim_FOLH1_level_prim_high+prim_low__"
            "prim_high_vs_prim_low_deseq_results_unique.csv"
        )
    ),
    index_col=0,
).set_index("SYMBOL")
diff_expr_df = diff_expr_df[diff_expr_df["padj"] < 0.05]

diff_meth_df = (
    pd.read_csv(
        (
            tcga_root.joinpath("minfi").joinpath(
                "sample_type_prim_FOLH1_level_prim_high+prim_low__"
                "diff_meth_probes_noob_quantile_top_table_prim_high_vs_prim_low_"
                "wrt_mean_diff_0_0_hg38_genes_promoters.csv"
            )
        ),
        index_col=0,
    )
    .replace("NA_character_", np.nan)
    .dropna(subset=["annot.symbol"])
)
diff_meth_df = (
    diff_meth_df[diff_meth_df["adj.P.Val"] < 0.05]
    .groupby("annot.symbol")
    .mean(numeric_only=True)
    .rename_axis("SYMBOL")
)

In [ ]:
common_genes = diff_expr_df.index.intersection(diff_meth_df.index)
print(len(common_genes))
starburst_df = pd.DataFrame()
starburst_df["expr_lfc"] = diff_expr_df.loc[common_genes, "log2FoldChange"]
starburst_df["meth_lfc"] = diff_meth_df.loc[common_genes, "logFC"]

In [ ]:
starburst_df.loc[
    (abs(starburst_df["expr_lfc"]) < 1) & (abs(starburst_df["meth_lfc"]) < 1), "label"
] = "Not relevant"

starburst_df.loc[
    (starburst_df["expr_lfc"] > 1) & (abs(starburst_df["meth_lfc"]) < 1), "label"
] = "Up regulated"
starburst_df.loc[
    (starburst_df["expr_lfc"] > 1) & (starburst_df["meth_lfc"] > 1), "label"
] = "Up regulated & Hyper methylated"
starburst_df.loc[
    (starburst_df["expr_lfc"] > 1) & (starburst_df["meth_lfc"] < -1), "label"
] = "Up regulated & Hypo methylated"

starburst_df.loc[
    (starburst_df["expr_lfc"] < -1) & (abs(starburst_df["meth_lfc"]) < 1), "label"
] = "Down regulated"
starburst_df.loc[
    (starburst_df["expr_lfc"] < -1) & (starburst_df["meth_lfc"] > 1), "label"
] = "Down regulated & Hyper methylated"
starburst_df.loc[
    (starburst_df["expr_lfc"] < -1) & (starburst_df["meth_lfc"] < -1), "label"
] = "Down regulated & Hypo methylated"

starburst_df.loc[
    (abs(starburst_df["expr_lfc"]) < 1) & (starburst_df["meth_lfc"] > 1), "label"
] = "Hyper methylated"
starburst_df.loc[
    (abs(starburst_df["expr_lfc"]) < 1) & (starburst_df["meth_lfc"] < -1), "label"
] = "Hypo methylated"

In [ ]:
starburst_df.to_excel(writer_2, sheet_name="2C. Starburst (PRIM)")

In [ ]:
prim_label_colors = {
    k: v for k, v in label_colors.items() if k in starburst_df["label"].unique()
}

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.scatterplot(
    data=starburst_df,
    x="expr_lfc",
    y="meth_lfc",
    hue="label",
    palette=prim_label_colors,
    hue_order=prim_label_colors.keys(),
    ax=ax,
    s=15,
)
highlight_labels = (
    "Up regulated & Hyper methylated",
    "Up regulated & Hypo methylated",
    "Down regulated & Hyper methylated",
    "Down regulated & Hypo methylated",
)

labels_counter = dict()
labels_text_y = defaultdict(lambda: 99)
for gene, meta in starburst_df.sort_values(
    ["expr_lfc", "meth_lfc"], key=abs, ascending=False
).iterrows():
    label = meta["label"]
    if label in highlight_labels:
        if labels_counter.get(label, 0) == 5:
            continue
        labels_counter[label] = labels_counter.get(label, 0) + 1

        text_y_new = meta["meth_lfc"] + 0.1
        text_y_new = (
            text_y_new
            if abs(labels_text_y[label] - text_y_new) > 0.1
            else (labels_text_y[label] + 0.1)
        )
        labels_text_y[label] = text_y_new

        plt.annotate(
            gene,
            xy=(meta["expr_lfc"], meta["meth_lfc"]),
            xytext=(meta["expr_lfc"] + 0.25, text_y_new),
            arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
            fontsize=14,
            weight="semibold",
        )

ymin, ymax = (
    starburst_df["meth_lfc"].min() - 0.5,
    starburst_df["meth_lfc"].max() + 0.5,
)
plt.ylim((ymin, ymax))

xmin, xmax = (
    starburst_df["expr_lfc"].min() - 0.5,
    starburst_df["expr_lfc"].max() + 0.5,
)
plt.xlim((xmin, xmax))

# vertical lines
plt.vlines(
    1,
    ymin=ymin,
    ymax=ymax,
    colors="grey",
    linestyles="dashed",
)
plt.vlines(
    -1,
    ymin=ymin,
    ymax=ymax,
    colors="grey",
    linestyles="dashed",
)

# horizontal lines
plt.hlines(
    1,
    xmin=xmin,
    xmax=xmax,
    colors="grey",
    linestyles="dashed",
)
plt.hlines(
    -1,
    xmin=xmin,
    xmax=xmax,
    colors="grey",
    linestyles="dashed",
)

# axis labels
plt.xlabel("Gene Expression ($Log_2$ Fold Change)", fontsize=20)
plt.ylabel("DNA Methylation ($Log_2$ Fold Change)", fontsize=20)
# plt.legend(title=r"$\bf Multi-omics\ relationship$", loc=(1.02, 0.3))
plt.legend(title=r"$\bf Multi-omics\ relationship$", loc="lower left")
# plt.suptitle(
#     "Multi-omics gene signatures (prim. high vs prim. low)",
#     fontsize=20,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3C_prim_starburst.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3C_prim_starburst.svg"), bbox_inches="tight")

### 3D: Barplot of GSEA (DE) gene sets for category H (prim over/under)

In [ ]:
gsea_h_df = pd.read_csv(
    pcta_wcdt_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(
        "sample_type_met+prim_FOLH1_level_prim_high+prim_low_"
        "_prim_high_vs_prim_low_gsea_H.csv"
    ),
    index_col=0,
)
gsea_h_df.index = [
    idx.replace("HALLMARK_", "").replace("_", " ") for idx in gsea_h_df.index
]

In [ ]:
gsea_h_df.to_excel(writer_2, sheet_name="2D. GSEA DE H (PRIM)")

In [ ]:
data = gsea_h_df["enrichmentScore"].sort_values(ascending=False).round(2).to_frame()

fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.barplot(data=data, x="enrichmentScore", y=data.index, palette=palette_str)
for container in ax.containers:
    ax.bar_label(container, padding=5)

ax.set_ylabel("Gene Sets", fontsize=20)
plt.yticks(fontsize=16)
ax.set_xlabel("Enrichment Score", fontsize=20)
ax.set_xlim([ax.get_xlim()[0] * 1.2, ax.get_xlim()[1] * 1.2])
# plt.title(
#     "GSEA (DE) Analysis on MSigDB H Collection (Prim. High vs Prim. low)",
#     fontsize=20,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3D_gsea_de_h_prim.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3D_gsea_de_h_prim.svg"), bbox_inches="tight")

### 3E: Barplot of GSEA (DM) gene sets for category H (prim over/under)

In [ ]:
gsea_h_df = pd.read_csv(
    tcga_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(
        "sample_type_prim_FOLH1_level_prim_high+prim_low_"
        "_diff_meth_probes_noob_quantile_top_table_prim_high_vs_prim_low_"
        "wrt_mean_diff_0_0_hg38_genes_promoters_gsea_H.csv"
    ),
    index_col=0,
)
gsea_h_df.index = [
    idx.replace("HALLMARK_", "").replace("_", " ") for idx in gsea_h_df.index
]

In [ ]:
gsea_h_df.to_excel(writer_2, sheet_name="2E. GSEA DM H (PRIM)")

In [ ]:
data = gsea_h_df["enrichmentScore"].sort_values(ascending=False).round(2).to_frame()

fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.barplot(data=data, x="enrichmentScore", y=data.index, palette=palette_str)
for container in ax.containers:
    ax.bar_label(container, padding=5)

ax.set_ylabel("Gene Sets", fontsize=20)
plt.yticks(fontsize=16)
ax.set_xlabel("Enrichment Score", fontsize=20)
ax.set_xlim([ax.get_xlim()[0] * 1.2, ax.get_xlim()[1] * 1.2])
# plt.title(
#     "GSEA (DM) Analysis on MSigDB H Collection (Prim. High vs Prim. low)",
#     fontsize=20,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3E_gsea_dm_h_prim.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3E_gsea_dm_h_prim.svg"), bbox_inches="tight")

### 3F: Starburst plot (met)

In [ ]:
diff_expr_df = pd.read_csv(
    (
        pcta_wcdt_root.joinpath("deseq2").joinpath(
            "sample_type_met+prim_FOLH1_level_met_high+met_low__"
            "met_high_vs_met_low_deseq_results_unique.csv"
        )
    ),
    index_col=0,
).set_index("SYMBOL")
diff_expr_df = diff_expr_df[diff_expr_df["padj"] < 0.05]

diff_meth_df = (
    pd.read_csv(
        (
            wcdt_root.joinpath("methylkit").joinpath(
                "sample_type_met_FOLH1_level_met_high+met_low__"
                "diff_meth_met_high_vs_met_low_ann_genes_hg38_genes_promoters.csv"
            )
        ),
        index_col=0,
    )
    .replace("NA_character_", np.nan)
    .dropna(subset=["annot.symbol"])
)
diff_meth_df = (
    diff_meth_df[diff_meth_df["qvalue"] < 0.05]
    .groupby("annot.symbol")
    .mean(numeric_only=True)
    .rename_axis("SYMBOL")
)

In [ ]:
common_genes = diff_expr_df.index.intersection(diff_meth_df.index)
print(len(common_genes))
starburst_df = pd.DataFrame()
starburst_df["expr_lfc"] = diff_expr_df.loc[common_genes, "log2FoldChange"]
starburst_df["meth_diff"] = diff_meth_df.loc[common_genes, "meth.diff"]

In [ ]:
meth_diff_th = 15

In [ ]:
starburst_df.loc[
    (abs(starburst_df["expr_lfc"]) < 1)
    & (abs(starburst_df["meth_diff"]) < meth_diff_th),
    "label",
] = "Not relevant"

starburst_df.loc[
    (starburst_df["expr_lfc"] > 1) & (abs(starburst_df["meth_diff"]) < meth_diff_th),
    "label",
] = "Up regulated"
starburst_df.loc[
    (starburst_df["expr_lfc"] > 1) & (starburst_df["meth_diff"] > meth_diff_th), "label"
] = "Up regulated & Hyper methylated"
starburst_df.loc[
    (starburst_df["expr_lfc"] > 1) & (starburst_df["meth_diff"] < -meth_diff_th),
    "label",
] = "Up regulated & Hypo methylated"

starburst_df.loc[
    (starburst_df["expr_lfc"] < -1) & (abs(starburst_df["meth_diff"]) < meth_diff_th),
    "label",
] = "Down regulated"
starburst_df.loc[
    (starburst_df["expr_lfc"] < -1) & (starburst_df["meth_diff"] > meth_diff_th),
    "label",
] = "Down regulated & Hyper methylated"
starburst_df.loc[
    (starburst_df["expr_lfc"] < -1) & (starburst_df["meth_diff"] < -meth_diff_th),
    "label",
] = "Down regulated & Hypo methylated"

starburst_df.loc[
    (abs(starburst_df["expr_lfc"]) < 1) & (starburst_df["meth_diff"] > meth_diff_th),
    "label",
] = "Hyper methylated"
starburst_df.loc[
    (abs(starburst_df["expr_lfc"]) < 1) & (starburst_df["meth_diff"] < -meth_diff_th),
    "label",
] = "Hypo methylated"

In [ ]:
starburst_df.to_excel(writer_2, sheet_name="2D. Starburst (MET)")

In [ ]:
met_label_colors = {
    k: v for k, v in label_colors.items() if k in starburst_df["label"].unique()
}

In [ ]:
fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.scatterplot(
    data=starburst_df,
    x="expr_lfc",
    y="meth_diff",
    hue="label",
    palette=met_label_colors,
    hue_order=met_label_colors.keys(),
    ax=ax,
    s=15,
)
highlight_labels = (
    "Up regulated & Hyper methylated",
    "Up regulated & Hypo methylated",
    "Down regulated & Hyper methylated",
    "Down regulated & Hypo methylated",
)

labels_counter = dict()
labels_text_y = defaultdict(lambda: 99)
for gene, meta in starburst_df.sort_values(
    ["expr_lfc", "meth_diff"], key=abs, ascending=False
).iterrows():
    label = meta["label"]
    if label in highlight_labels:
        if labels_counter.get(label, 0) == 5:
            continue
        labels_counter[label] = labels_counter.get(label, 0) + 1

        text_y_new = meta["meth_diff"] - 1.5
        text_y_new = (
            text_y_new
            if abs(labels_text_y[label] - text_y_new) > 1.5
            else (labels_text_y[label] - 1.5)
        )
        labels_text_y[label] = text_y_new

        plt.annotate(
            gene,
            xy=(meta["expr_lfc"], meta["meth_diff"]),
            xytext=(meta["expr_lfc"] + 0.5, text_y_new),
            arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
            fontsize=14,
            weight="semibold",
        )

ymin, ymax = (
    starburst_df["meth_diff"].min() - 0.5,
    starburst_df["meth_diff"].max() + 0.5,
)
plt.ylim((ymin, ymax))

xmin, xmax = (
    starburst_df["expr_lfc"].min() - 0.5,
    starburst_df["expr_lfc"].max() + 0.5,
)
plt.xlim((xmin, xmax))

# vertical lines
plt.vlines(
    1,
    ymin=ymin,
    ymax=ymax,
    colors="grey",
    linestyles="dashed",
)
plt.vlines(
    -1,
    ymin=ymin,
    ymax=ymax,
    colors="grey",
    linestyles="dashed",
)

# horizontal lines
plt.hlines(
    meth_diff_th,
    xmin=xmin,
    xmax=xmax,
    colors="grey",
    linestyles="dashed",
)
plt.hlines(
    -meth_diff_th,
    xmin=xmin,
    xmax=xmax,
    colors="grey",
    linestyles="dashed",
)

# axis labels
plt.xlabel("Gene Expression ($Log_2$ Fold Change)", fontsize=20)
plt.ylabel("DNA Methylation (Methylation difference)", fontsize=20)
# plt.legend(title=r"$\bf Multi-omics\ relationship$", loc=(1.02, 0.3))
plt.legend(title=r"$\bf Multi-omics\ relationship$", loc="lower left")
# plt.suptitle(
#     "Multi-omics gene signatures (MET. high vs MET. low)",
#     fontsize=20,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3F_met_starburst.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3F_met_starburst.svg"), bbox_inches="tight")

### 3G: Barplot of GSEA (DE) gene sets for category H (met over/under)

In [ ]:
gsea_h_df = pd.read_csv(
    pcta_wcdt_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(
        "sample_type_met+prim_FOLH1_level_met_high+met_low_"
        "_met_high_vs_met_low_gsea_H.csv"
    ),
    index_col=0,
)
gsea_h_df.index = [
    idx.replace("HALLMARK_", "").replace("_", " ") for idx in gsea_h_df.index
]

In [ ]:
gsea_h_df.to_excel(writer_2, sheet_name="2G. GSEA DE H (MET)")

In [ ]:
data = gsea_h_df["enrichmentScore"].sort_values(ascending=False).round(2).to_frame()

fig, ax = plt.subplots(figsize=(14, 10), dpi=100)

ax = sns.barplot(data=data, x="enrichmentScore", y=data.index, palette=palette_str)
for container in ax.containers:
    ax.bar_label(container, padding=5)

ax.set_ylabel("Gene Sets", fontsize=20)
plt.yticks(fontsize=16)
ax.set_xlabel("Enrichment Score", fontsize=20)
ax.set_xlim([ax.get_xlim()[0] * 1.2, ax.get_xlim()[1] * 1.2])
# plt.title(
#     "GSEA (DE) Analysis on MSigDB H Collection (Met. High vs Met. low)",
#     fontsize=20,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("3G_gsea_de_h_met.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("3G_gsea_de_h_met.svg"), bbox_inches="tight")

### 3H: Barplot of GSEA (DM) gene sets for category H (met over/under)

**No results available!**

In [ ]:
writer_2.close()

## Figure 4: Identifying PSMA protein community

This figure summarizes the sets of top genes found and the PPI network analysis.

1. (**A**) Jointplot of z-scores vs. log2 fold changes of DEGs (prim over)
2. (**B**) Jointplot of z-scores vs. log2 fold changes of DMGs (prim over)
3. (**C**) Jointplot of z-scores vs. log2 fold changes of DEGs (prim under)
4. (**D**) Jointplot of z-scores vs. log2 fold changes of DMGs (prim under)
5. (**E**) Jointplot of z-scores vs. log2 fold changes of DEGs (met over)
6. (**F**) Jointplot of z-scores vs. log2 fold changes of DMGs (met over)
7. (**G**) Jointplot of z-scores vs. log2 fold changes of DEGs (met under)
8. (**H**) Jointplot of z-scores vs. log2 fold changes of DMGs (met under)

In [ ]:
writer_3 = pd.ExcelWriter(pcta_wcdt_add_files_path / "additional_file_3.xlsx")

In [ ]:
ymin, ymax = (-1, 12)
xmin, xmax = (-8, 8)

### 4A. Jointplot of z-scores vs. log2 fold changes of DEGs (prim over)

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)
print(top_genes_df.shape)

In [ ]:
top_genes_df.to_excel(writer_3, sheet_name="3A. Top genes stats (PRIM over)")

In [ ]:
x, y, hue = ["diff_expr_log2FoldChange", "mean_z_score", "Statistical significance"]
expr_scatter_df = top_genes_df[[x, y]].dropna()

expr_scatter_df[hue] = "Significant"
expr_scatter_df.loc[expr_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
expr_scatter_df.loc[abs(expr_scatter_df[x]) < 1, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=expr_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] > 1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] < -1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 2, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 3, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DEGs (Prim over)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel(r"$Log_{2}$ Fold Change", fontsize=14)
plt.legend(loc="upper left")
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4A_jointplot_diff_expr_prim_over.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4A_jointplot_diff_expr_prim_over.svg"),
    bbox_inches="tight",
)

### 4B. Jointplot of z-scores vs. log2 fold changes of DMGs (prim over)

In [ ]:
x, y, hue = ["diff_meth_promoters_logFC", "mean_z_score", "Statistical significance"]
meth_scatter_df = top_genes_df[[x, y]].dropna()

meth_scatter_df[hue] = "Significant"
meth_scatter_df.loc[meth_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
meth_scatter_df.loc[abs(meth_scatter_df[x]) < 1, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=meth_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] > 1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] < -1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 2, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 3, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )


plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DMGs (Prim over)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel(r"$Log_{2}$ Fold Change", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4B_jointplot_diff_meth_prim_over.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4B_jointplot_diff_meth_prim_over.svg"),
    bbox_inches="tight",
)

### 4C. Jointplot of z-scores vs. log2 fold changes of DEGs (prim under)

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)
print(top_genes_df.shape)

In [ ]:
top_genes_df.to_excel(writer_3, sheet_name="3B. Top genes stats (PRIM under)")

In [ ]:
x, y, hue = ["diff_expr_log2FoldChange", "mean_z_score", "Statistical significance"]
expr_scatter_df = top_genes_df[[x, y]].dropna()

expr_scatter_df[hue] = "Significant"
expr_scatter_df.loc[expr_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
expr_scatter_df.loc[abs(expr_scatter_df[x]) < 1, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=expr_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] > 1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] < -1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 2, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 3, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )


plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DEGs (Prim under)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel(r"$Log_{2}$ Fold Change", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4C_jointplot_diff_expr_prim_under.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4C_jointplot_diff_expr_prim_under.svg"),
    bbox_inches="tight",
)

### 4D. Jointplot of z-scores vs. log2 fold changes of DMGs (prim under)

In [ ]:
x, y, hue = ["diff_meth_promoters_logFC", "mean_z_score", "Statistical significance"]
meth_scatter_df = top_genes_df[[x, y]].dropna()

meth_scatter_df[hue] = "Significant"
meth_scatter_df.loc[meth_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
meth_scatter_df.loc[abs(meth_scatter_df[x]) < 1, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=meth_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] > 1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] < -1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 2, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 3, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )


plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DMGs (Prim under)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel(r"$Log_{2}$ Fold Change", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4D_jointplot_diff_meth_prim_under.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4D_jointplot_diff_meth_prim_under.svg"),
    bbox_inches="tight",
)

### 4E. Jointplot of z-scores vs. log2 fold changes of DEGs (met over)

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)
print(top_genes_df.shape)

In [ ]:
top_genes_df.to_excel(writer_3, sheet_name="3C. Top genes stats (MET over)")

In [ ]:
x, y, hue = ["diff_expr_log2FoldChange", "mean_z_score", "Statistical significance"]
expr_scatter_df = top_genes_df[[x, y]].dropna()

expr_scatter_df[hue] = "Significant"
expr_scatter_df.loc[expr_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
expr_scatter_df.loc[abs(expr_scatter_df[x]) < 1, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=expr_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] > 1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] < -1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 2, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 3, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )


plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DEGs (Met over)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel(r"$Log_{2}$ Fold Change", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4E_jointplot_diff_expr_met_over.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4E_jointplot_diff_expr_met_over.svg"),
    bbox_inches="tight",
)

### 4F. Jointplot of z-scores vs. log2 fold changes of DMGs (met over)

In [ ]:
x, y, hue = [
    "diff_meth_promoters_meth.diff",
    "mean_z_score",
    "Statistical significance",
]
meth_scatter_df = top_genes_df[[x, y]].dropna()

meth_scatter_df[hue] = "Significant"
meth_scatter_df.loc[meth_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
meth_scatter_df.loc[abs(meth_scatter_df[x]) < 10, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=meth_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] > 1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] < -1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 5, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 15, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

plt.xlim(-40, 40)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(10, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-10, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DMGs (Met over)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel("Mean Methylation Difference", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4F_jointplot_diff_meth_met_over.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4F_jointplot_diff_meth_met_over.svg"),
    bbox_inches="tight",
)

### 4G. Jointplot of z-scores vs. log2 fold changes of DEGs (met under)

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)
print(top_genes_df.shape)

In [ ]:
top_genes_df.to_excel(writer_3, sheet_name="3D. Top genes stats (MET under)")

In [ ]:
x, y, hue = ["diff_expr_log2FoldChange", "mean_z_score", "Statistical significance"]
expr_scatter_df = top_genes_df[[x, y]].dropna()

expr_scatter_df[hue] = "Significant"
expr_scatter_df.loc[expr_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
expr_scatter_df.loc[abs(expr_scatter_df[x]) < 1, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=expr_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] > 1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    expr_scatter_df.loc[
        (expr_scatter_df[x] < -1) & (expr_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 2, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 3, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )


plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-1, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DEGs (Met under)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel(r"$Log_{2}$ Fold Change", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4G_jointplot_diff_expr_met_under.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4G_jointplot_diff_expr_met_under.svg"),
    bbox_inches="tight",
)

### 4H. Jointplot of z-scores vs. log2 fold changes of DMGs (met under)

In [ ]:
x, y, hue = [
    "diff_meth_promoters_meth.diff",
    "mean_z_score",
    "Statistical significance",
]
meth_scatter_df = top_genes_df[[x, y]].dropna()

meth_scatter_df[hue] = "Significant"
meth_scatter_df.loc[meth_scatter_df["mean_z_score"] < 1.96, hue] = "Not Significant"
meth_scatter_df.loc[abs(meth_scatter_df[x]) < 5, hue] = "Not Significant"

palette = {"Significant": "#8B3A3A", "Not Significant": "#BFBFBF"}

fig, ax = plt.subplots(figsize=(6, 5), dpi=100)

grid = sns.scatterplot(data=meth_scatter_df, x=x, y=y, hue=hue, palette=palette, ax=ax)

top_up = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] > 1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)
top_down = (
    meth_scatter_df.loc[
        (meth_scatter_df[x] < -1) & (meth_scatter_df[hue] == "Significant")
    ]
    .sort_values(y, ascending=False)
    .head(5)
)

text_y = 99
for gene, gene_meta in top_up.iterrows():
    text_y_new = gene_meta[y] + 0.75
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 0.75
        else (text_y := text_y - 0.75)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] + 7, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

text_y = 99
for gene, gene_meta in top_down.iterrows():
    text_y_new = gene_meta[y] + 1
    text_y_new = (
        (text_y := text_y_new)
        if text_y - text_y_new > 1
        else (text_y := text_y - 1)
    )
    plt.annotate(
        gene,
        xy=(gene_meta[x], gene_meta[y]),
        xytext=(gene_meta[x] - 15, text_y_new),
        arrowprops=dict(facecolor="black", width=2, headwidth=5, headlength=5),
        fontsize=12,
    )

plt.xlim(-40, 40)
plt.ylim(ymin, ymax)
plt.legend(loc="upper left")
plt.vlines(10, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.vlines(-10, ymin=ymin, ymax=ymax, colors="#8C8C8C", linestyles="dashed")
plt.hlines(1.96, xmin=xmin, xmax=xmax, colors="#8C8C8C", linestyles="dashed")
# plt.suptitle("Gene Occurrence of DMGs (Met under)", y=1.03)
plt.ylabel("Gene occurrence (Z-score normalised)", fontsize=14)
plt.xlabel("Mean Methylation Difference", fontsize=14)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4H_jointplot_diff_meth_met_under.pdf"),
    bbox_inches="tight",
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("4H_jointplot_diff_meth_met_under.svg"),
    bbox_inches="tight",
)

In [ ]:
writer_3.close()

## Figure 5: Functional profiling of PSMA protein community

1. (**A**) Barplot & Stripplots of ORA results for category H (prim over)
2. (**B**) Barplot & Stripplots of ORA results for category H (prim under)
3. (**C**) Barplot & Stripplots of ORA results for category H (met over)
4. (**D**) Barplot & Stripplots of ORA results for category H (met under)

In [ ]:
writer_4 = pd.ExcelWriter(pcta_wcdt_add_files_path / "additional_file_4.xlsx")

In [ ]:
ytick_length = 36

### 5A: Barplot & Stripplots of ORA results for category H (prim over)

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]

ora_h_file = (
    clustering_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(f"{psma_cluster_id}_ora_H.csv")
)

if ora_h_file.exists():
    ora_h_df = (
        pd.read_csv(ora_h_file, index_col=0)
        .drop(columns=["ID", "Description"])
        .sort_values("Count", ascending=False)
    )

ora_h_df.index = ora_h_df.index.str.replace("HALLMARK_", "").str.replace("_", " ")

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)

num_cols = ["mean_z_score", "diff_expr_log2FoldChange", "diff_meth_promoters_logFC"]
data_df = (
    ora_h_df["geneID"]
    .str.split("/")
    .explode()
    .rename("gene")
    .to_frame()
    .join(top_genes_df[num_cols], on="gene")
)

In [ ]:
data_df.to_excel(writer_4, sheet_name="4A. ORA H Genes (PRIM over)")

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(24, 3), dpi=100)

# 0. Count barplot
sns.barplot(data=ora_h_df, x="Count", y=ora_h_df.index, ax=ax[0], palette=palette_str)
for container in ax[0].containers:
    ax[0].bar_label(container, padding=5)

# ax[0].set_ylabel("Gene Sets", fontsize=14)
ax[0].set_xlabel("Count", fontsize=18)
ax[0].set_yticklabels(
    [
        " " * (ytick_length - len(gene_set)) + gene_set
        for gene_set in ora_h_df.index.tolist()
    ],
    fontsize=16,
)
ax[0].set_xlim([ax[0].get_xlim()[0] * 1.2, ax[0].get_xlim()[1] * 1.2])

# 1. Mean Z-score stripplot
data_df_plot = deepcopy(data_df.dropna(subset=["mean_z_score"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    ax=ax[1],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    hue=data_df_plot.index,
    ax=ax[1],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[1].get_legend().remove()
ax[1].set_xlim([0, ax[1].get_xlim()[1] * 1.2])
ax[1].set_yticklabels([])
ax[1].set_xlabel("Mean Z-score", fontsize=18)

# 2. Diff. Expr. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_expr_log2FoldChange"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    ax=ax[2],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    hue=data_df_plot.index,
    ax=ax[2],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[2].get_legend().remove()
ax[2].set_xlim([ax[2].get_xlim()[0] * 1.2, ax[2].get_xlim()[1] * 1.2])
ax[2].set_yticklabels([])
ax[2].set_xlabel("Diff. Expr. $Log_2$ Fold Change", fontsize=18)

# 3. Diff. Meth. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_meth_promoters_logFC"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_logFC",
    ax=ax[3],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_logFC",
    hue=data_df_plot.index,
    ax=ax[3],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[3].get_legend().remove()
ax[3].set_xlim([ax[3].get_xlim()[0] * 1.2, ax[3].get_xlim()[1] * 1.2])
ax[3].set_yticklabels([])
ax[3].set_xlabel("Diff. Meth. $Log_2$ Fold Change", fontsize=18)

plt.subplots_adjust(wspace=0.08, hspace=0)
# plt.suptitle(
#     "Over-representation analysis on Hallmark gene sets\n(Prim. High vs Prim. low, over)",
#     fontsize=24,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("5A_ora_h_prim_over.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("5A_ora_h_prim_over.svg"), bbox_inches="tight")

In [ ]:
data_df[
    (abs(data_df["diff_expr_log2FoldChange"]) > 1)
    | (abs(data_df["diff_meth_promoters_logFC"]) > 1)
].sort_values("diff_expr_log2FoldChange", key=abs, ascending=False).drop_duplicates(
    "gene"
)

### 5B: Barplot & Stripplots of ORA results for category H (prim under)

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]

ora_h_file = (
    clustering_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(f"{psma_cluster_id}_ora_H.csv")
)

if ora_h_file.exists():
    ora_h_df = (
        pd.read_csv(ora_h_file, index_col=0)
        .drop(columns=["ID", "Description"])
        .sort_values("Count", ascending=False)
    )

ora_h_df.index = ora_h_df.index.str.replace("HALLMARK_", "").str.replace("_", " ")

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)

num_cols = ["mean_z_score", "diff_expr_log2FoldChange", "diff_meth_promoters_logFC"]
data_df = (
    ora_h_df["geneID"]
    .str.split("/")
    .explode()
    .rename("gene")
    .to_frame()
    .join(top_genes_df[num_cols], on="gene")
)

In [ ]:
data_df.to_excel(writer_4, sheet_name="4B. ORA H Genes (PRIM under)")

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(24, 2.75), dpi=100)

# 0. Count barplot
sns.barplot(data=ora_h_df, x="Count", y=ora_h_df.index, ax=ax[0], palette=palette_str)
for container in ax[0].containers:
    ax[0].bar_label(container, padding=5)

# ax[0].set_ylabel("Gene Sets", fontsize=14)
ax[0].set_xlabel("Count", fontsize=18)
ax[0].set_yticklabels(
    [
        " " * (ytick_length - len(gene_set)) + gene_set
        for gene_set in ora_h_df.index.tolist()
    ],
    fontsize=16,
)
ax[0].set_xlim([ax[0].get_xlim()[0] * 1.2, ax[0].get_xlim()[1] * 1.2])

# 1. Mean Z-score stripplot
data_df_plot = deepcopy(data_df.dropna(subset=["mean_z_score"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    ax=ax[1],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    hue=data_df_plot.index,
    ax=ax[1],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[1].get_legend().remove()
ax[1].set_xlim([0, ax[1].get_xlim()[1] * 1.2])
ax[1].set_yticklabels([])
ax[1].set_xlabel("Mean Z-score", fontsize=18)

# 2. Diff. Expr. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_expr_log2FoldChange"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    ax=ax[2],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    hue=data_df_plot.index,
    ax=ax[2],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[2].get_legend().remove()
ax[2].set_xlim([ax[2].get_xlim()[0] * 1.2, ax[2].get_xlim()[1] * 1.2])
ax[2].set_yticklabels([])
ax[2].set_xlabel("Diff. Expr. $Log_2$ Fold Change", fontsize=18)

# 3. Diff. Meth. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_meth_promoters_logFC"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_logFC",
    ax=ax[3],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_logFC",
    hue=data_df_plot.index,
    ax=ax[3],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[3].get_legend().remove()
ax[3].set_xlim([ax[3].get_xlim()[0] * 1.2, ax[3].get_xlim()[1] * 1.2])
ax[3].set_yticklabels([])
ax[3].set_xlabel("Diff. Meth. $Log_2$ Fold Change", fontsize=18)

plt.subplots_adjust(wspace=0.08, hspace=0)
# plt.suptitle(
#     "Over-representation analysis on Hallmark gene sets\n(Prim. High vs Prim. low, under)",
#     fontsize=24,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(
    pcta_wcdt_figs_path.joinpath("5B_ora_h_prim_under.pdf"), bbox_inches="tight"
)
plt.savefig(
    pcta_wcdt_figs_path.joinpath("5B_ora_h_prim_under.svg"), bbox_inches="tight"
)

In [ ]:
data_df[
    (abs(data_df["diff_expr_log2FoldChange"]) > 1)
    | (abs(data_df["diff_meth_promoters_logFC"]) > 1)
].sort_values("diff_expr_log2FoldChange", key=abs, ascending=False).drop_duplicates(
    "gene"
)

### 5C: Barplot & Stripplots of ORA results for category H (met over)

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]

ora_h_file = (
    clustering_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(f"{psma_cluster_id}_ora_H.csv")
)

if ora_h_file.exists():
    ora_h_df = (
        pd.read_csv(ora_h_file, index_col=0)
        .drop(columns=["ID", "Description"])
        .sort_values("Count", ascending=False)
    )

ora_h_df.index = ora_h_df.index.str.replace("HALLMARK_", "").str.replace("_", " ")

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)

num_cols = ["mean_z_score", "diff_expr_log2FoldChange", "diff_meth_promoters_meth.diff"]
data_df = (
    ora_h_df["geneID"]
    .str.split("/")
    .explode()
    .rename("gene")
    .to_frame()
    .join(top_genes_df[num_cols], on="gene")
)

In [ ]:
data_df.to_excel(writer_4, sheet_name="4C. ORA H Genes (MET over)")

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(24, 1.75), dpi=100)

# 0. Count barplot
sns.barplot(data=ora_h_df, x="Count", y=ora_h_df.index, ax=ax[0], palette=palette_str)
for container in ax[0].containers:
    ax[0].bar_label(container, padding=5)

ax[0].set_xlabel("Count", fontsize=18)
ax[0].set_yticklabels(
    [
        " " * (ytick_length - len(gene_set)) + gene_set
        for gene_set in ora_h_df.index.tolist()
    ],
    fontsize=16,
)
ax[0].set_xlim([ax[0].get_xlim()[0] * 1.2, ax[0].get_xlim()[1] * 1.2])

# 1. Mean Z-score stripplot
data_df_plot = deepcopy(data_df.dropna(subset=["mean_z_score"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    ax=ax[1],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    hue=data_df_plot.index,
    ax=ax[1],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[1].get_legend().remove()
ax[1].set_xlim([0, ax[1].get_xlim()[1] * 1.2])
ax[1].set_yticklabels([])
ax[1].set_xlabel("Mean Z-score", fontsize=18)

# 2. Diff. Expr. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_expr_log2FoldChange"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    ax=ax[2],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    hue=data_df_plot.index,
    ax=ax[2],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[2].get_legend().remove()
ax[2].set_xlim([ax[2].get_xlim()[0] * 1.2, ax[2].get_xlim()[1] * 1.2])
ax[2].set_yticklabels([])
ax[2].set_xlabel("Diff. Expr. $Log_2$ Fold Change", fontsize=18)

# 3. Diff. Meth. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_meth_promoters_meth.diff"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_meth.diff",
    ax=ax[3],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_meth.diff",
    hue=data_df_plot.index,
    ax=ax[3],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[3].get_legend().remove()
ax[3].set_xlim([ax[3].get_xlim()[0] * 1.2, ax[3].get_xlim()[1] * 1.2])
ax[3].set_yticklabels([])
ax[3].set_xlabel("Diff. Meth. Difference", fontsize=18)

plt.subplots_adjust(wspace=0.08, hspace=0)
# plt.suptitle(
#     "Over-representation analysis on Hallmark gene sets\n(Met. High vs Met. low, over)",
#     fontsize=24,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("5C_ora_h_met_over.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("5C_ora_h_met_over.svg"), bbox_inches="tight")

In [ ]:
data_df[
    (abs(data_df["diff_expr_log2FoldChange"]) > 1)
    | (abs(data_df["diff_meth_promoters_meth.diff"]) > 10)
].sort_values("diff_expr_log2FoldChange", key=abs, ascending=False).drop_duplicates(
    "gene"
)

### 5D: Barplot & Stripplots of ORA results for category H (met under)

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]

ora_h_file = (
    clustering_root.joinpath("functional")
    .joinpath("MSIGDB")
    .joinpath(f"{psma_cluster_id}_ora_H.csv")
)

if ora_h_file.exists():
    ora_h_df = (
        pd.read_csv(ora_h_file, index_col=0)
        .drop(columns=["ID", "Description"])
        .sort_values("Count", ascending=False)
    )

ora_h_df.index = ora_h_df.index.str.replace("HALLMARK_", "").str.replace("_", " ")

In [ ]:
top_genes_df = pd.read_csv(
    pcta_wcdt_root.joinpath("degss_genes_stats")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("gene_occurrence_stats_bootstrap_z_score_summary_ann.csv"),
    index_col=0,
)

num_cols = ["mean_z_score", "diff_expr_log2FoldChange", "diff_meth_promoters_meth.diff"]
data_df = (
    ora_h_df["geneID"]
    .str.split("/")
    .explode()
    .rename("gene")
    .to_frame()
    .join(top_genes_df[num_cols], on="gene")
)

In [ ]:
data_df.to_excel(writer_4, sheet_name="4D. ORA H Genes (MET under)")

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(24, 4), dpi=100)

# 0. Count barplot
sns.barplot(data=ora_h_df, x="Count", y=ora_h_df.index, ax=ax[0], palette=palette_str)
for container in ax[0].containers:
    ax[0].bar_label(container, padding=5)

ax[0].set_xlabel("Count", fontsize=18)
ax[0].set_yticklabels(
    [
        " " * (ytick_length - len(gene_set)) + gene_set
        for gene_set in ora_h_df.index.tolist()
    ],
    fontsize=16,
)
ax[0].set_xlim([ax[0].get_xlim()[0] * 1.2, ax[0].get_xlim()[1] * 1.2])

# 1. Mean Z-score stripplot
data_df_plot = deepcopy(data_df.dropna(subset=["mean_z_score"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    ax=ax[1],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="mean_z_score",
    hue=data_df_plot.index,
    ax=ax[1],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[1].get_legend().remove()
ax[1].set_xlim([0, ax[1].get_xlim()[1] * 1.2])
ax[1].set_yticklabels([])
ax[1].set_xlabel("Mean Z-score", fontsize=18)

# 2. Diff. Expr. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_expr_log2FoldChange"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    ax=ax[2],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_expr_log2FoldChange",
    hue=data_df_plot.index,
    ax=ax[2],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[2].get_legend().remove()
ax[2].set_xlim([ax[2].get_xlim()[0] * 1.2, ax[2].get_xlim()[1] * 1.2])
ax[2].set_yticklabels([])
ax[2].set_xlabel("Diff. Expr. $Log_2$ Fold Change", fontsize=18)

# 3. Diff. Meth. LFC
data_df_plot = deepcopy(data_df.dropna(subset=["diff_meth_promoters_meth.diff"]))
sns.boxplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_meth.diff",
    ax=ax[3],
    palette=palette_str,
    boxprops=dict(alpha=0.5),
)
sns.stripplot(
    data=data_df_plot,
    y=data_df_plot.index,
    x="diff_meth_promoters_meth.diff",
    hue=data_df_plot.index,
    ax=ax[3],
    palette=palette_str,
    marker="o",
    linewidth=1,
    jitter=0.2,
)
ax[3].get_legend().remove()
ax[3].set_xlim([ax[3].get_xlim()[0] * 1.2, ax[3].get_xlim()[1] * 1.2])
ax[3].set_yticklabels([])
ax[3].set_xlabel("Diff. Meth. Difference", fontsize=18)

plt.subplots_adjust(wspace=0.08, hspace=0)
# plt.suptitle(
#     "Over-representation analysis on Hallmark gene sets\n(Met. High vs Met. low, under)",
#     fontsize=24,
#     y=1.02,
# )
plt.tight_layout()
plt.savefig(pcta_wcdt_figs_path.joinpath("5D_ora_h_met_under.pdf"), bbox_inches="tight")
plt.savefig(pcta_wcdt_figs_path.joinpath("5D_ora_h_met_under.svg"), bbox_inches="tight")

In [ ]:
data_df[
    (abs(data_df["diff_expr_log2FoldChange"]) > 1)
    | (abs(data_df["diff_meth_promoters_meth.diff"]) > 10)
].sort_values("diff_expr_log2FoldChange", key=abs, ascending=False).drop_duplicates(
    "gene"
)

In [ ]:
writer_4.close()

## Additional information and figures

### Table 1: Enriched gene sets from different methods

#### Primary

In [ ]:
prim_dgsva_files = sorted(
    list(
        pcta_wcdt_root.joinpath("diff_gsva").glob(
            "**/*prim_high_vs_prim_low_top_table_padj_0_05_up_0_0.csv"
        )
    )
    + list(
        pcta_wcdt_root.joinpath("diff_gsva").glob(
            "**/*prim_high_vs_prim_low_top_table_padj_0_05_down_0_0.csv"
        )
    )
)

prim_dgsva_counts = dict()
for dgsva_file in prim_dgsva_files:
    msigdb_cat = dgsva_file.parent.stem
    lfc_level = (
        "under-enriched"
        if dgsva_file.stem.split("_")[-3] == "down"
        else "over-enriched"
    )
    prim_dgsva_counts[(msigdb_cat, lfc_level)] = len(
        pd.read_csv(dgsva_file, index_col=0)
    )

In [ ]:
prim_gsea_de = sorted(
    pcta_wcdt_root.joinpath("functional")
    .joinpath("MSIGDB")
    .glob("*_prim_high_vs_prim_low_gsea_*.csv")
)

prim_gsea_de_counts = dict()
for gsea_file in prim_gsea_de:
    msigdb_cat = gsea_file.stem.split("_")[-1]
    df = pd.read_csv(gsea_file, index_col=0)
    prim_gsea_de_counts[(msigdb_cat, "over-enriched")] = len(
        df[df["enrichmentScore"] > 0]
    )
    prim_gsea_de_counts[(msigdb_cat, "under-enriched")] = len(
        df[df["enrichmentScore"] < 0]
    )

In [ ]:
prim_gsea_dm = sorted(
    tcga_root.joinpath("functional")
    .joinpath("MSIGDB")
    .glob("*_prim_high_vs_prim_low_wrt_mean_diff_0_0_hg38_genes_promoters_gsea_*.csv")
)

prim_gsea_dm_counts = dict()
for gsea_file in prim_gsea_dm:
    msigdb_cat = gsea_file.stem.split("_")[-1]
    df = pd.read_csv(gsea_file, index_col=0)
    prim_gsea_dm_counts[(msigdb_cat, "over-enriched")] = len(
        df[df["enrichmentScore"] > 0]
    )
    prim_gsea_dm_counts[(msigdb_cat, "under-enriched")] = len(
        df[df["enrichmentScore"] < 0]
    )

#### Metastatic

In [ ]:
met_dgsva_files = sorted(
    list(
        pcta_wcdt_root.joinpath("diff_gsva").glob(
            "**/*met_high_vs_met_low_top_table_padj_0_05_up_0_0.csv"
        )
    )
    + list(
        pcta_wcdt_root.joinpath("diff_gsva").glob(
            "**/*met_high_vs_met_low_top_table_padj_0_05_down_0_0.csv"
        )
    )
)

met_dgsva_counts = dict()
for dgsva_file in met_dgsva_files:
    msigdb_cat = dgsva_file.parent.stem
    lfc_level = (
        "under-enriched"
        if dgsva_file.stem.split("_")[-3] == "down"
        else "over-enriched"
    )
    met_dgsva_counts[(msigdb_cat, lfc_level)] = len(
        pd.read_csv(dgsva_file, index_col=0)
    )

In [ ]:
met_gsea_de = sorted(
    pcta_wcdt_root.joinpath("functional")
    .joinpath("MSIGDB")
    .glob("*_met_high_vs_met_low_gsea_*.csv")
)

met_gsea_de_counts = dict()
for gsea_file in met_gsea_de:
    msigdb_cat = gsea_file.stem.split("_")[-1]
    df = pd.read_csv(gsea_file, index_col=0)
    met_gsea_de_counts[(msigdb_cat, "over-enriched")] = len(
        df[df["enrichmentScore"] > 0]
    )
    met_gsea_de_counts[(msigdb_cat, "under-enriched")] = len(
        df[df["enrichmentScore"] < 0]
    )

In [ ]:
met_gsea_dm = sorted(
    wcdt_root.joinpath("methylkit")
    .joinpath("functional")
    .joinpath("MSIGDB")
    .glob("*_met_high_vs_met_low_ann_genes_hg38_genes_promoters_gsea_*.csv")
)

met_gsea_dm_counts = dict()
for gsea_file in met_gsea_dm:
    msigdb_cat = gsea_file.stem.split("_")[-1]
    df = pd.read_csv(gsea_file, index_col=0)
    met_gsea_dm_counts[(msigdb_cat, "over-enriched")] = len(
        df[df["enrichmentScore"] > 0]
    )
    met_gsea_dm_counts[(msigdb_cat, "under-enriched")] = len(
        df[df["enrichmentScore"] < 0]
    )

#### Build table

In [ ]:
table_1_df = (
    pd.DataFrame(
        {
            ("Prim high/low", "DGSVA"): prim_dgsva_counts,
            ("Prim high/low", "GSEA DE"): prim_gsea_de_counts,
            ("Prim high/low", "GSEA DM"): prim_gsea_dm_counts,
            ("Met high/low", "DGSVA"): met_dgsva_counts,
            ("Met high/low", "GSEA DE"): met_gsea_de_counts,
            ("Met high/low", "GSEA DM"): met_gsea_dm_counts,
        }
    )
    .fillna(0)
    .astype(int)
)
table_1_df.index.set_names(["MSigDB Category", "Enrichment Level"], inplace=True)
table_1_df.to_csv(pcta_wcdt_add_files_path / "table_1.csv")
table_1_df

In [ ]:
table_1_df.to_latex(
    pcta_wcdt_add_files_path / "table_1.tex",
    bold_rows=True,
    caption="Number of (differentially) enriched gene sets for each MSigDB category and enrichment method.",
    label="table1",
)

### Table 2: Cluster sizes

In [ ]:
writer_3 = pd.ExcelWriter(pcta_wcdt_add_files_path / "additional_file_3.xlsx")

In [ ]:
cluster_sizes = dict()

#### Prim over

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]
print("Cluster ID FOLH1 is in:", psma_cluster_id)

In [ ]:
clusters_df.to_excel(writer_3, sheet_name="3E. PPI Clustering (PRIM over)")

In [ ]:
cluster_sizes["prim_over"] = (
    clusters_df.groupby("cluster_id")["FOLH1_distance"].count().to_dict()
)

#### Prim under

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "prim_high_vs_prim_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]
print("Cluster ID FOLH1 is in:", psma_cluster_id)

In [ ]:
clusters_df.to_excel(writer_3, sheet_name="3F. PPI Clustering (PRIM under)")

In [ ]:
cluster_sizes["prim_under"] = (
    clusters_df.groupby("cluster_id")["FOLH1_distance"].count().to_dict()
)

#### Met over

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_up_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]
print("Cluster ID FOLH1 is in:", psma_cluster_id)

In [ ]:
clusters_df.to_excel(writer_3, sheet_name="3G. PPI Clustering (MET over)")

In [ ]:
cluster_sizes["met_over"] = (
    clusters_df.groupby("cluster_id")["FOLH1_distance"].count().to_dict()
)

#### Met under

In [ ]:
clustering_root = (
    pcta_wcdt_root.joinpath("degss_ppi_networks")
    .joinpath(
        "met_high_vs_met_low_degss_padj_0_05_down_0_0_"
        "gsea_H_C1_C2_C3_C4_C5_C6_C7_C8_union"
    )
    .joinpath("functional_network_at_500_score")
    .joinpath("clustering")
    .joinpath("p_1_0_q_0_5")
)

clusters_df = pd.read_csv(
    clustering_root.joinpath("genes_cluster_ids_metadata.csv"), index_col=0
).sort_values("cluster_id", ascending=True)
psma_cluster_id = clusters_df.loc["FOLH1"]["cluster_id"]
print("Cluster ID FOLH1 is in:", psma_cluster_id)

In [ ]:
clusters_df.to_excel(writer_3, sheet_name="3H. PPI Clustering (MET under)")
writer_3.close()

In [ ]:
cluster_sizes["met_under"] = (
    clusters_df.groupby("cluster_id")["FOLH1_distance"].count().to_dict()
)

#### Build table

In [ ]:
cluster_sizes_df = (
    pd.DataFrame(cluster_sizes).fillna(0).astype(int).rename_axis("Cluster ID")
)
cluster_sizes_df.to_csv(pcta_wcdt_add_files_path / "table_2.csv")
cluster_sizes_df

In [ ]:
cluster_sizes_df.to_latex(
    pcta_wcdt_add_files_path / "table_2.tex",
    bold_rows=True,
    caption="Size of clusters after ensemble clustering using Node2Vec embeddings.",
    label="table2",
)

### How many genes have both hypo- and hyper-methylated CpGs?

#### Primary

In [ ]:
diff_meth_df = (
    pd.read_csv(
        (
            tcga_root.joinpath("minfi").joinpath(
                "sample_type_prim_FOLH1_level_prim_high+prim_low__"
                "diff_meth_probes_noob_quantile_top_table_prim_high_vs_prim_low_"
                "wrt_mean_diff_0_0_hg38_genes_promoters.csv"
            )
        ),
        index_col=0,
    )
    .replace("NA_character_", np.nan)
    .dropna(subset=["annot.symbol"])
)

hypo_hyper_meth_counts = (
    diff_meth_df.groupby("annot.symbol")["logFC"]
    .apply(lambda x: pd.Series([(x < 0).sum(), (x >= 0).sum()]))
    .unstack()
    .rename(columns={0: "Hypo-methylated", 1: "Hyper-methylated"})
)

n_homogeneous = hypo_hyper_meth_counts[
    (hypo_hyper_meth_counts["Hypo-methylated"] == 0)
    | (hypo_hyper_meth_counts["Hyper-methylated"] == 0)
]
print("Number of genes with only hypo or hyper methylated CpGs:", len(n_homogeneous))
print(
    "Number of genes with both hypo or hyper methylated CpGs:",
    len(hypo_hyper_meth_counts) - len(n_homogeneous),
)

#### Metastatic

In [ ]:
diff_meth_df = (
    pd.read_csv(
        (
            wcdt_root.joinpath("methylkit").joinpath(
                "sample_type_met_FOLH1_level_met_high+met_low__"
                "diff_meth_met_high_vs_met_low_ann_genes_hg38_genes_promoters.csv"
            )
        ),
        index_col=0,
    )
    .replace("NA_character_", np.nan)
    .dropna(subset=["annot.symbol"])
)

hypo_hyper_meth_counts = (
    diff_meth_df.groupby("annot.symbol")["meth.diff"]
    .apply(lambda x: pd.Series([(x < 0).sum(), (x >= 0).sum()]))
    .unstack()
    .rename(columns={0: "Hypo-methylated", 1: "Hyper-methylated"})
)

n_homogeneous = hypo_hyper_meth_counts[
    (hypo_hyper_meth_counts["Hypo-methylated"] == 0)
    | (hypo_hyper_meth_counts["Hyper-methylated"] == 0)
]
print("Number of genes with only hypo or hyper methylated CpGs:", len(n_homogeneous))
print(
    "Number of genes with both hypo or hyper methylated CpGs:",
    len(hypo_hyper_meth_counts) - len(n_homogeneous),
)

In [ ]:
diff_meth_df["annot.symbol"].nunique()